In [2]:
import trio
import httpx

In [17]:
async def timer() -> None: 
    i = 1 
    while True: 
        await trio.sleep(1)
        if i ==1: 
            print(f"{i} second has passed")
        else: 
            print(f"{i} seconds have passed")
        i+=1 

Q4: Given this code snippet, what would be the order of what is printed?

In [18]:
async def get_user_id(user_id: int) -> dict:
    await trio.sleep(1)
    if user_id % 1 == 0: 
        return {"id": user_id, "username": f"User {user_id}"}
    else: 
        raise ValueError(f"User {user_id} not found")

async def get_user_data(user_ids: list[int]) -> dict[dict]:
    results = {}
    
    async def _fetch_and_store(user_id): 
        try: 
            result = await get_user_id(user_id)
            results[user_id] = {"status": "success", "data": result}
        except ValueError as e: 
            results[user_id] = {"status": "error", "error": str(e)}

    async with trio.open_nursery() as nursery: 
        for user_id in user_ids: 
            nursery.start_soon(_fetch_and_store, user_id)
    
    return results

In [16]:
import trio
import time 
async def child1(): 
    await trio.sleep(0.5)

async def child2(): 
    await trio.sleep(1.5)

start_time = time.time() 
async def parent():
    print(f"parent: started! {time.time()-start_time}s")
    async with trio.open_nursery() as nursery:
        await child1() 
        await child2()
        print("parent: waiting for children to finish...")
        # -- we exit the nursery block here --
    print(f"parent: all done! {time.time()-start_time}s")
trio.run(parent)

parent: started! 0.0008308887481689453s
parent: waiting for children to finish...
parent: all done! 2.0038490295410156s


In [12]:
import trio
async def get_lemon(): 
    print("Getting lemon")
    await trio.sleep(1)
    result = "lemon"
    return result

async def get_apple(): 
    print("Getting apple")
    await trio.sleep(0.5)
    result = "apple"
    return result

async def get_orange(): 
    print("Getting orange")
    await trio.sleep(0.25)
    result = "orange"
    return result

async def main(): 
    async with trio.open_nursery() as nursery: 
        output = nursery.start_soon(get_lemon)
        output = nursery.start_soon(get_apple)
        output = nursery.start_soon(get_orange)
    print("Task complete")
    print(output)
    
trio.run(main)

Getting lemon
Getting apple
Getting orange
Task complete
None


In [ ]:
import trio
async def foo(): 
    print("Start")
    await trio.sleep(1)
    print("End")

async def main(): 
    with trio.open_nursery() as nursery: 
        nursery.start_soon(foo)
    print("Task complete")

trio.run(main)

RuntimeError: use 'async with open_nursery(...)', not 'with open_nursery(...)'

In [ ]:
def example(): 
    print("Start")
    await trio.sleep(1)
    print("Middle")
    await trio.sleep(1)
    print("End")

trio.run(example)

In [8]:
import time
import trio

async def broken_double_sleep(x):
    print("*yawn* Going to sleep")
    start_time = time.perf_counter()

    trio.sleep(2 * x)

    sleep_time = time.perf_counter() - start_time
    print(f"Woke up after {sleep_time} seconds!")

trio.run(broken_double_sleep, 3)

*yawn* Going to sleep
Woke up after 0.00 seconds, feeling well rested!


/var/folders/jc/ctcxyx_55yqb7m0dg7sj4clm0000gn/T/ipykernel_33457/3884429731.py:8: RuntimeWarning: coroutine 'sleep' was never awaited
  trio.sleep(2 * x)


In [ ]:
import trio
async def example(): 
    await trio.sleep(1)
    result = "Hello World"
    return result

In [ ]:
import trio
async def example(): 
    await trio.sleep(1)
    print("Lemon")
    await trio.sleep(1)
    print("Orange")

async def main(): 
    async with trio.open_nursery() as nursery: 
        nursery.start_soon(example)
        print("Apple")

trio.run(main)

Apple
Lemon
Orange


In [ ]:
async def example(): 
    await trio.sleep(1)
    print("Lemon")
    await trio.sleep(1)
    print("Orange")

async def main(): 
    async with trio.open_nursery() as nursery: 
        nursery.start_soon(example)
        print("Apple")

trio.run(example)

Lemon
Orange


Trace the execution flow of this code and identify when each task runs and completes:

In [5]:
import trio
async def task_with_subtasks():
    print("Parent: starting")
    async with trio.open_nursery() as subtask_nursery:
        subtask_nursery.start_soon(subtask, 1)
        print("Parent: spawned first subtask")
        await trio.sleep(0.5)
        subtask_nursery.start_soon(subtask, 2)
        print("Parent: spawned second subtask")
    print("Parent: subtasks complete")

async def subtask(number):
    print(f"Subtask {number}: starting")
    await trio.sleep(1)
    print(f"Subtask {number}: finished")

trio.run(task_with_subtasks)

Parent: starting
Parent: spawned first subtask
Subtask 1: starting
Parent: spawned second subtask
Subtask 2: starting
Subtask 1: finished
Subtask 2: finished
Parent: subtasks complete


What will this code output and why? Pay attention to the behavior of the Semaphore:

In [4]:
async def worker(id, semaphore):
    print(f"Worker {id} waiting for semaphore")
    async with semaphore:
        print(f"Worker {id} acquired semaphore")
        await trio.sleep(1)
        print(f"Worker {id} releasing semaphore")

async def main():
    semaphore = trio.Semaphore(2)
    
    async with trio.open_nursery() as nursery:
        for i in range(1, 4):
            nursery.start_soon(worker, i, semaphore)
            await trio.sleep(0.1)
            
    print("All workers completed")

trio.run(main)

Worker 1 waiting for semaphore
Worker 1 acquired semaphore
Worker 2 waiting for semaphore
Worker 2 acquired semaphore
Worker 3 waiting for semaphore
Worker 1 releasing semaphore
Worker 3 acquired semaphore
Worker 2 releasing semaphore
Worker 3 releasing semaphore
All workers completed


What will be the output of this code, and why?

In [4]:
import trio
async def task(n):
    if n == 2:
        raise ValueError("Task 2 fails")
    await trio.sleep(0.5)
    print(f"Task {n} ending")

async def main():
    async with trio.open_nursery() as nursery:
        for i in [1, 2, 3]:
            nursery.start_soon(task, i)
    print("Main complete")

trio.run(main)

  + Exception Group Traceback (most recent call last):
  |   File "/Users/jhs/Code/AFP/skill-dev/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3549, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/jc/ctcxyx_55yqb7m0dg7sj4clm0000gn/T/ipykernel_16537/2163059941.py", line 14, in <module>
  |     trio.run(main)
  |     ~~~~~~~~^^^^^^
  |   File "/Users/jhs/Code/AFP/skill-dev/.venv/lib/python3.13/site-packages/trio/_core/_run.py", line 2423, in run
  |     raise runner.main_task_outcome.error
  |   File "/var/folders/jc/ctcxyx_55yqb7m0dg7sj4clm0000gn/T/ipykernel_16537/2163059941.py", line 9, in main
  |     async with trio.open_nursery() as nursery:
  |                ~~~~~~~~~~~~~~~~~^^
  |   File "/Users/jhs/Code/AFP/skill-dev/.venv/lib/python3.13/site-packages/trio/_core/_run.py", line 1058, in __aexit__
  |     raise combined_error_from_nursery
  | Except

Trace the execution of this code with race conditions. What are the possible outputs?

In [2]:
# shared_counter = 0

async def increment_counter(id, n_times):
    global shared_counter
    for _ in range(n_times):
        current = shared_counter
        await trio.sleep(0.01)  # Simulate some work
        shared_counter = current + 1
        print(f"Task {id} incremented to {shared_counter}")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(increment_counter, 1, 3)
        nursery.start_soon(increment_counter, 2, 3)
    
    print(f"Final counter value: {shared_counter}")

trio.run(main)

Task 1 incremented to 1
Task 2 incremented to 1
Task 2 incremented to 2
Task 1 incremented to 2
Task 2 incremented to 3
Task 1 incremented to 3
Final counter value: 3


In [6]:
# Review the following code snippet and identify the bug -> does not execute async bc of the await
import trio
async def process_item(item):
    return item * 2

async def process_data(items):
    results = []
    async with trio.open_nursery() as nursery:
        for item in items:
            result = await process_item(item)
            results.append(result)
    return results

In [5]:
# Review the code and describe the bug: -> task never completes because of timeout function
async def run_with_timeout():
    with trio.move_on_after(1):
        print("Starting task")
        await long_running_task()

async def long_running_task():
    await trio.sleep(2)
    print("Task completed")

trio.run(run_with_timeout)

Starting task


In [21]:
# What's the problem with this code? -> client is created each time and never closed
import httpx 

async def fetch_data_repeatedly():
    i = 1 
    while i < 3:
        client = httpx.AsyncClient()
        try:
            response = await client.get("https://api.worldbank.org/v2/country/ARB/indicator/NY.GDP.MKTP.CD?date=2019&format=json")
            if response.status_code == 200:
                print("Service is up!")
            else:
                print(f"Service returned: {response.status_code}")
        except Exception as e:
            print(f"Error occurred: {e}")
        
        await trio.sleep(3)
        i+=1

trio.run(fetch_data_repeatedly)

Service is up!
Service is up!


In [13]:
# what is wrong with this code? where is the await missing? 
import trio
async def fetch_data(user_id):
    await trio.sleep(1)
    return {'id': user_id, 'data': f'User data for {user_id}'}

async def process_users(user_ids):
    all_data = {}
    
    async def process_one_user(uid):
        data = fetch_data(uid) 
        all_data[uid] = data
    
    async with trio.open_nursery() as nursery:
        for uid in user_ids:
            nursery.start_soon(process_one_user, uid)
    
    return all_data

trio.run(process_users, [1, 2, 3])

{1: <coroutine object fetch_data at 0x10eca7e80>,
 2: <coroutine object fetch_data at 0x10eca7b80>,
 3: <coroutine object fetch_data at 0x10eca7c40>}

In [9]:
# what is wrong with this code? # nursery not properly open 
import trio 
async def process_task(result_dict):
    await trio.sleep(2)
    result_dict['status'] = 'completed'

async def main():
    results = {}
    
    nursery = trio.open_nursery()
    nursery.start_soon(process_task, results)
    
    # Do some other work
    await trio.sleep(1)
    print(f"Results: {results}")  # Expect to see status='completed'

trio.run(main)

AttributeError: 'NurseryManager' object has no attribute 'start_soon'

### Implementation Question:

In [ ]:
# A 
import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

async def main():
    await say_hello()
    await say_world()

trio.run(main)

# B

import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(say_hello)
        nursery.start_soon(say_world)
    print("Task complete")

trio.run(main)

import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(say_hello)
        nursery.start_soon(say_world)
        print("Task complete")

trio.run(main)

# C
import trio

def say_hello():
    trio.sleep(1)
    print("Hello")

def say_world():
    trio.sleep(2)
    print("World")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(say_hello)
        nursery.start_soon(say_world)

trio.run(main)

# D
import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

def main():
    nursery = trio.open_nursery()
    nursery.start_soon(say_hello)
    nursery.start_soon(say_world)

trio.run(main)

In [6]:
async def download_and_save_text(client, url, filename):
    print(f"Downloading {filename}...")
    try:
        response = await client.get(url)
        response.raise_for_status()
        
        with open(f"downloads/{filename}.txt", "w") as f:
            f.write(response.text)
        
        print(f"Finished downloading {filename}")
    except httpx.HTTPError as e:
        print(f"Error downloading {filename}: {e}")

async def task3():
    urls = [
        ("https://www.gutenberg.org/files/2600/2600-0.txt", "War and Peace"),
        ("https://www.gutenberg.org/files/1342/1342-0.txt", "Pride and Prejudice"),
        ("https://www.gutenberg.org/files/1661/1661-0.txt", "The Adventures of Sherlock Holmes")
    ]
    
    async with httpx.AsyncClient() as client:
        async with trio.open_nursery() as nursery:
            for url, name in urls:
                nursery.start_soon(download_and_save_text, client, url, name)

trio.run(task3)

Finished downloading The Adventures of Sherlock Holmes
Finished downloading Pride and Prejudice
Finished downloading War and Peace


In [2]:
def fetch_record(record_id: int) -> str:
    if record_id > 100: 
        return f"Record-{record_id}"
    else: 
        raise KeyError(f"Record {record_id} not found in database")

#A
def process_records(id_list):
    records = {}
    for record_id in id_list:
        data = fetch_record(record_id)
        records[record_id] = data
    return records

#B
def process_records(id_list):
    records = {}
    for record_id in id_list:
        try:
            data = fetch_record(record_id)
        except ValueError:
            data = 'not_found'
        records[record_id] = data
    return records

#C 
def process_records(id_list):
    records = {}
    for record_id in id_list:
        try:
            data = fetch_record(record_id)
        except:
            data = 'not_found'
        records[record_id] = data
    return records

#D
def process_records(id_list):
    records = {}
    for record_id in id_list:
        if record_id <= 100:
            records[record_id] = 'not_found'
        else:
            records[record_id] = fetch_record(record_id)
    return records


{1: 'not_found', 3: 'not_found', 101: 'Record-101'}

In [ ]:
# Q: Which of the following correctly downloads and saves a text file from a URL using the requests library?

#A 
import requests
def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

#B
import requests
def download_file(url, filename):
    response = requests.put(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

#C
import requests
def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'w') as file:
        file.write(response)
#D
import requests
def download_file(url, filename):
    with open(filename, 'wb') as file:
        file.write(requests.content(url))